In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [2]:
data = pd.read_csv('_Dữ_liệu_giao_dịch_ngày__202406152152.csv')
data['Ngày'] = pd.to_datetime(data['Ngày'])

In [3]:
features = ['Mở cửa', 'Đóng cửa', 'Cao nhất', 'Thấp nhất', 'Trung bình', 'GD khớp lệnh KL']
data = data[['Ngày', 'Mã CK'] + features]
data = data.sort_values(by=['Mã CK', 'Ngày'])

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
data[features] = scaler.fit_transform(data[features])

In [5]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [6]:
def create_sequences(data, seq_length, features):
    sequences = []
    for stock in data['Mã CK'].unique():
        stock_data = data[data['Mã CK'] == stock][features].values
        for i in range(len(stock_data) - seq_length):
            sequences.append(stock_data[i:i+seq_length])
    return np.array(sequences)
seq_length = 60
sequences = create_sequences(data, seq_length, features)
X = sequences[:, :-1, :]
y = sequences[:, -1, features.index('Đóng cửa')]

In [7]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [8]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(seq_length-1, len(features))))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
3074/3074 ━━━━━━━━━━━━━━━━━━━━ 117s 37ms/step - loss: 3.5941e-04 - val_loss: 3.7055e-05
Epoch 2/5
3074/3074 ━━━━━━━━━━━━━━━━━━━━ 113s 37ms/step - loss: 9.7798e-05 - val_loss: 3.0991e-05
Epoch 3/5
3074/3074 ━━━━━━━━━━━━━━━━━━━━ 110s 36ms/step - loss: 8.5122e-05 - val_loss: 7.1032e-05
Epoch 4/5
3074/3074 ━━━━━━━━━━━━━━━━━━━━ 108s 35ms/step - loss: 8.2816e-05 - val_loss: 2.3697e-05
Epoch 5/5
3074/3074 ━━━━━━━━━━━━━━━━━━━━ 108s 35ms/step - loss: 8.0539e-05 - val_loss: 2.4497e-05


In [9]:
model.save('lstm_model.h5')

In [10]:
def predict_next_close(stock_data, seq_length, model, features, scaler):
    last_sequence = stock_data[features].values[-seq_length:]
    last_sequence = scaler.transform(last_sequence)
    last_sequence = np.expand_dims(last_sequence[:-1], axis=0)
    predicted_price = model.predict(last_sequence)
    predicted_price = np.concatenate([predicted_price, np.zeros((predicted_price.shape[0], len(features)-1))], axis=1)
    predicted_price = scaler.inverse_transform(predicted_price)
    return predicted_price[0][features.index('Đóng cửa')]

In [12]:
profits = {}
for stock in data['Mã CK'].unique():
    stock_data = data[data['Mã CK'] == stock]
    current_price = stock_data['Đóng cửa'].values[-1]
    predicted_price = predict_next_close(stock_data, seq_length, model, features, scaler)
    profit = (predicted_price - current_price) / current_price
    profits[stock] = profit

# In kết quả
print(profits)

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
{'AAA': 28.08777777777778, 'AAM': 37.53937665782493, 'AAT': 90.95787974683546, 'ABR': 23.235771476230195, 'ABS': 66.73587412587412, 'ABT': 7.5744142814989655, 'ACB': 11.37066411238825, 'ACC': 20.461366322008864, 'ACG': 5.005102293862368, 'ACL': 23.857733105218134, 'ADG': 17.287407174323473, 'ADP': 9.45652752788773, 'ADS': 21.719851446442533, 'AGG': 10.86553287055941, 'AGM': 79.49498614958449, 'AGR': 13.646517137096774, 'AMD': inf, 'ANV': 8.112163687676386, 'APC': 47.51200333889817, 'APG': 22.45334140435835, 'APH': 34.87492592592592, 'ASG': 14.801353996737355, 'ASM': 26.31079887218045, 'ASP': 79.94342618384398, 'AST': 3.7801760157920707, 'BAF': 13.147366114897762, 'BBC': 4.9194723976369925, 'BCE': 59.038615702479326, 'BCG': 36.78763328998699, 'BCM': 3.7104376722321284, 'BFC': 7.44974992730445, 'BHN': 6.510646161798914, 'BIC': 7.902784926470587, 'BID': 5.263998706617805, 'BKG': 90.66779179810723, 'BMC': 16.838360957642728, 'BMI': 11.062552926525528, 

In [13]:
best_stock = max(profits, key=profits.get)
print(f"Mã cổ phiếu có lợi nhuận dự đoán cao nhất là: {best_stock} với lợi nhuận dự đoán là: {profits[best_stock]*100:.2f}%")

# Giả sử bạn có một số tiền để đầu tư
investment_amount = 1000000  # Số tiền đầu tư
potential_profit = investment_amount * profits[best_stock]
print(f"Lợi nhuận dự đoán khi đầu tư vào {best_stock} là: {potential_profit:.2f} VND")

Mã cổ phiếu có lợi nhuận dự đoán cao nhất là: AMD với lợi nhuận dự đoán là: inf%
Lợi nhuận dự đoán khi đầu tư vào AMD là: inf VND
